## 서울시에서 공개한 코로나19 발생동향 분석
* http://www.seoul.go.kr/coronaV/coronaStatus.do

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_name = f"seoul_covid_6_25_.csv"
file_name

In [ ]:
df = pd.read_csv(file_name)
df.shape

In [ ]:
df = df.sort_values(["연번"], ascending=False)

In [ ]:
df.head()

In [ ]:
df.tail()

## 시각화 도구 불러오기

In [ ]:
# matplotlib.pyplot 을 통해 한글폰트를 설정합니다.
import matplotlib.pyplot as plt

plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)

### 시각화 선명하게 설정하기

In [ ]:
# retina 디스플레이가 지원되는 환경에서 시각화의 폰트가 좀 더 선명해 보입니다.
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("retina")

## 거주지별 확진자

In [ ]:
# 거주지(구별) 확진자의 빈도수를 구하고 시각화 합니다.
gu_count = df["거주지"].value_counts()
gu_count.head()

In [ ]:
# 구별 확진자의 수를 시각화 합니다.
gu_count.sort_values().plot.barh(figsize=(10, 10), grid=True)

In [ ]:
# 서울에서 확진판정을 받은 데이터이기 때문에 거주지가 서울이 아닐 수도 있습니다.
# 거주지 별로 서울시에 해당되는 데이터만 따로 가져옵니다.
gu = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']
gu[:5]

In [ ]:
# 거주지가 서울이 아닌 지역을 따로 추출합니다.
set(gu_count.index) - set(gu)

In [ ]:
# 구를 전처리 하기 쉽게 컬럼으로 변환하기 위해 reset_index 로 변환합니다.
df_gu = gu_count.reset_index()
df_gu.columns = ["구", "환자수"]
df_gu.head()

In [ ]:
df

In [ ]:
# 서울에서 확진 받은 사람 중 서울 vs 타지역을 비교해 보기 위해
# "지역"이라는 새로운 컬럼을 만들어 서울지역이 아니라면 "타지역" 이라는 값을 넣어줍니다. 
# .loc[행인덱스]
# .loc[행, 열]
# .loc[조건, 열]
df.loc[df["거주지"].isin(gu), "지역"] = df["거주지"]
df.loc[~df["거주지"].isin(gu), "지역"] = "타지역"
df.head()

In [ ]:
# 위의 방법으로 할수도 있고 아래의 방법으로 만들수도 있습니다.
# 함수 혹은 익명함수를 사용하는 방법으로 "타지역" 값을 만들 수도 있습니다.
df["지역"] = df["거주지"].map(lambda x : x if x in gu else "타지역")
df[["거주지", "지역"]].head()

In [ ]:
df_gu_etc = df["지역"].value_counts()
df_gu_etc

In [ ]:
g = df_gu_etc.plot.barh(figsize=(10, 7))
g.axvline(50, linestyle=":", color="green")

## 확진일

In [ ]:
# 확진일의 빈도수를 봅니다. 어느 날짜에 가장 많이 확진이 되었는지 봅니다.
df["확진일"].value_counts()

In [ ]:
# "확진일" 컬럼의 데이터가 날짜 형태가 아니라 문자형태 입니다.
df["확진일"].head(1)

In [ ]:
# 데이터 타입을 변경해서 날짜형태로 변환합니다.
# 판다스의 to_datetime 을 사용해서 날짜 타입으로 변경할 수 있습니다.
# 연도가 없기 때문에 2020년을 날짜에 추가하고 "-" 문자로 날짜를 연결해 줍니다.
df["확진일자"] = pd.to_datetime("2020"+ "-" + df["확진일"].str.replace(".", "-"))
df["확진일자"].head()

In [ ]:
df["확진일자"].value_counts().sort_index().plot(figsize=(15, 4), rot=30, grid=True)
plt.axhline(30, linestyle=":")

In [ ]:
# 일자별 확진자수를 선그래프로 그립니다.
# 연도는 모두 2020년이기 때문에 월일만 표기되도록 슬라이싱을 사용해 "월일" 컬럼을 만듭니다.
df["월일"] = df["확진일자"].astype(str).map(lambda x : x[-5:])
day_count = df["월일"].value_counts().sort_index()
g = day_count.plot(figsize=(15, 4), rot=30)


for i in range(len(day_count)):
    case_count = day_count.iloc[i] 
    if case_count > 10:
        g.text(x=i-0.3, y=day_count.iloc[i], s=day_count.iloc[i])

In [ ]:
day_count = df["월일"].value_counts().sort_index()
day_count.iloc[0]

In [ ]:
g = day_count.plot.bar(figsize=(24, 4))
g.axhline(30, linestyle=":")

for i in range(len(day_count)):
    case_count = day_count.iloc[i] 
    if case_count > 10:
        g.text(x=i-0.3, y=day_count.iloc[i], s=day_count.iloc[i])

## 모든 날짜를 행에 만들어 주기
* 확진자가 없는 날의 데이터도 만들어 줍니다.

In [ ]:
# 첫 확진일 부터 마지막 확진일까지 가져옵니다.
# 데이터프레임의 첫번째 날짜는 first_day 에 마지막 날짜는 last_day 에 담습니다.

first_day = df.iloc[-1]["확진일자"]
last_day = df.iloc[0]["확진일자"]
first_day, last_day

In [ ]:
# pd.date_range 를 통해 시작하는 날짜부터 끝나는 날짜까지의 DatetimeIndex 를 만들고 days 라는 변수에 저장합니다.
days = pd.date_range(first_day, last_day)
days[:5]

In [ ]:
# days 변수의 값으로 "연월일" 이라는 컬럼이름을 갖는 데이터프레임을 만듭니다.
days = pd.DataFrame({"확진일자": days})
days.head()

In [ ]:
all_day = days.merge(cum, left_on="확진일자", right_on=cum.index, how="left")
all_day = all_day.rename(columns={"연번":"확진수"})
all_day.head()

## 누적 확진자 수 구하기

In [ ]:
all_day["누적확진"] = all_day["확진수"].fillna(0).cumsum()
all_day

In [ ]:
all_day["일자"] = all_day["확진일자"].astype(str).map(lambda x: x[-5:])
all_day["일자"].head()

In [ ]:
cum_day = all_day.set_index("일자")
cum_day = cum_day[["확진수", "누적확진"]]
cum_day

In [ ]:
cum_day.plot(figsize=(15, 4))

In [ ]:
cum_day["누적확진"].plot(figsize=(15, 4))
cum_day["확진수"].plot(figsize=(15, 4))

In [ ]:
cum_day["확진수"].plot(figsize=(15, 4))
cum_day["확진수"].plot.bar(figsize=(15, 4))

In [ ]:
cum_day["누적확진"].plot(figsize=(15, 4))

In [ ]:
all_day["확진월"] = all_day["확진일자"].dt.month
all_day["확진요일"] = all_day["확진일자"].dt.dayofweek
all_day.head()

## 요일별 확진 수 

In [ ]:
all_day_week = all_day.groupby(["확진월", "확진요일"])["확진수"].sum().unstack()
all_day_week.style.background_gradient(cmap='PuBu')

## 접촉력

In [ ]:
df["접촉력"].value_counts()

In [ ]:
df["접촉경로"] = df["접촉력"]
df.loc[df["접촉력"].str.contains("접촉자"), "접촉경로"] = "접촉자"
df.loc[df["접촉력"].str.contains("은평성모"), "접촉경로"] = "은평성모병원"
df.loc[df["접촉력"].str.contains("가족"), "접촉경로"] = "가족"
df.loc[df["접촉력"].str.contains("부동산"), "접촉경로"] = "부동산중개법인"
df.loc[df["접촉력"].str.contains("확잍중"), "접촉경로"] = "확인중"

In [ ]:
df["접촉경로"].value_counts()

## 가장 많은 전파가 일어난 번호

In [ ]:
import re

re.sub("[^0-9]", "", "#7265 접촉(추정)")

In [ ]:
def get_number(text):
    return re.sub("[^0-9]", "", text)
    
get_number("#7265 접촉(추정)")

In [ ]:
df["접촉번호"] = df["접촉력"].map(get_number)
contact = df["접촉번호"].value_counts().reset_index()
contact = contact.drop(0)
contact.columns = ["환자번호", "전파수"]
contact.head(10)

In [ ]:
top_contact = contact.merge(df, left_on="환자번호", right_on="환자")
top_contact.head()

In [ ]:
top_contactor = top_contact.iloc[0, 0]
top_contactor

In [ ]:
df[df["접촉경로"].str.contains(top_contactor)]

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
df_call = df[df["접촉경로"].str.contains("콜센터")]
print(df_call.shape)
df_call["접촉경로"].value_counts()

## 조치사항

In [ ]:
# 조치사항에 대한 빈도수를 세어봅니다.
# value_counts 는 Series 에만 사용할 수 있습니다.
# 단일 변수의 빈도수를 세는데 사용합니다.
df["조치사항"].value_counts()

In [ ]:
df["퇴원"] = df["조치사항"].str.contains("퇴원")
df["병원"] = df["조치사항"].str.replace("\(퇴원\)", "")

In [ ]:
df["퇴원"].value_counts()

In [ ]:
df["퇴원"].value_counts(normalize=True)

In [ ]:
df["병원"].value_counts()

## 퇴원여부

In [ ]:
df["퇴원"].value_counts()

## 여행력

In [ ]:
df["해외"] = df["여행력"]
df["해외"] = df["해외"].str.strip()
df["해외"] = df["해외"].replace("-", np.nan)
df["해외"] = df["해외"].replace("", np.nan)
df["해외"].value_counts()

In [ ]:
df_oversea = df[df["해외"].notnull()].copy()
df_oversea.shape

In [ ]:
df_oversea["해외"] = df_oversea["해외"].replace("우한교민", "우한 교민")
df_oversea["해외"].value_counts()

In [ ]:
europe = "체코, 헝가리, 오스트리아, 이탈리아, 프랑스, 모로코, 독일, 스페인, 영국, 폴란드"
europe = europe.replace(", ", "|")
europe

In [ ]:
df_oversea.loc[df_oversea["해외"].str.contains(europe), "해외"] = "유럽"
df_oversea["해외"].value_counts()

In [ ]:
day_oversea = df_oversea.groupby(["확진일자", "해외"])["연번"].count().groupby(level=[1]).cumsum()
day_oversea.head()

In [ ]:
day_oversea = day_oversea.reset_index()
day_oversea = day_oversea.rename(columns={'연번':'확진자수'})
print(day_oversea.shape)
day_oversea.head(10)

In [ ]:
oversea_count = df_oversea["해외"].value_counts(ascending=True)
oversea_count

In [ ]:
oversea_count.plot.barh()

In [ ]:
day_oversea.iloc[-1, 0]

In [ ]:
for i, val in enumerate(oversea_count[-5:]):
    num = -5 + i
    print(day_oversea["확진일자"].nunique(), i, num, val)

In [ ]:
oversea_count.index[-1]

In [ ]:
day_oversea.head()

In [ ]:
df_oversea = day_oversea.set_index("확진일자")
df_oversea.head()

In [ ]:
plt.figure(figsize=(15, 4))
g = day_oversea[].plot.bar()
g.legend(bbox_to_anchor=(1.3, 1))
last_day = day_oversea.iloc[-1, 0]

for i, val in enumerate(oversea_count[-5:]):
    i_num = -5 + i
    g.text(x=last_day, y=val, s=f"{oversea_count.index[i_num]} {val}")

In [ ]:
df_oversea["구"] = df_oversea["거주지"]
df_oversea.loc[~df_oversea["거주지"].str.endswith("구"), "구"] = "타지역"

In [ ]:
oversea_count = df_oversea["구"].value_counts()
oversea_count

In [ ]:
df["구"] = df["거주지"]
df.loc[~df["거주지"].str.endswith("구"), "구"] = "타지역"
df.loc[df["거주지"] == "대구", "구"] = "타지역"

In [ ]:
gu_count = df["구"].value_counts()

In [ ]:
plt.figure(figsize=(10, 8))
plt.title("해외유입 전체 구별 확진자")
gu_count.sort_values().plot.barh()

In [ ]:
plt.figure(figsize=(20, 4))
plt.title("해외입국자 구별 확진자")
sns.countplot(data=df_oversea, x="구", order=oversea_count.index, palette="Oranges_r")

In [ ]:
plt.figure(figsize=(15, 4))
sns.barplot(x=gu_count.index, y=gu_count, palette="Blues_r")
sns.barplot(x=oversea_count.index, y=oversea_count, palette="Oranges_r")

In [ ]:
plt.figure(figsize=(20, 4))
sns.countplot(data=df_oversea, x="구", hue="성별", order=gu_count.index)

In [ ]:
plt.figure(figsize=(20, 4))
sns.countplot(data=df_oversea, x="구", hue="퇴원", order=gu_count.index)

In [ ]:
df_oversea.loc[df_oversea["구"].isin(["강남구", "서초구", "송파구"]), "해외"].value_counts()

In [ ]:
from wordcloud import WordCloud

def wordcloud(data, width=1200, height=600):
    word_draw = WordCloud(font_path='/Library/Fonts/NanumBarunGothic.ttf',
                                    width = width, height = height, 
                                    background_color="#FFFFFF",
                                    random_state=42,
                                   ).generate(data)

    plt.figure(figsize=(15,10))
    plt.imshow(word_draw)
    plt.axis("off")
    plt.show() 

In [ ]:
wordcloud(" ".join(df_oversea["여행력"]))

In [ ]:
wordcloud(" ".join(df_oversea["해외"]))